In [29]:
import pandas as pd
import numpy as np
import nltk
import re
import unicodedata
from nltk.corpus import stopwords

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB

nltk.download('stopwords')


[nltk_data] Downloading package stopwords to /home/gitpod/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [7]:
df_raw = pd.read_csv("https://raw.githubusercontent.com/4GeeksAcademy/machine-learning-content/master/assets/spam.csv")

In [8]:
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Category  5572 non-null   object
 1   Message   5572 non-null   object
dtypes: object(2)
memory usage: 87.2+ KB


In [9]:
df_raw.sample(10)

,Category,Message
1398,ham,Then we wait 4 u lor... No need 2 feel bad lar...
383,ham,Hey leave it. not a big deal:-) take care.
1591,ham,Somewhr someone is surely made 4 u. And God ha...
841,ham,I luv u soo much u dont understand how specia...
1873,ham,Oh ok i didnt know what you meant. Yep i am ba...
1215,ham,I'll text now! All creepy like so he won't thi...
4547,ham,Never try alone to take the weight of a tear t...
5561,ham,Get me out of this dump heap. My mom decided t...
1353,ham,Let Ur Heart Be Ur Compass Ur Mind Ur Map Ur S...
3717,ham,Networking technical support associate.


In [11]:
df_raw['Message'] = df_raw['Message'].str.lower() #convierto a minuscula

In [12]:
df_raw['Message'].str.split(expand=True).stack().value_counts()[:60] #corto el texto en pedacitos, asi lo veo por unidad para hacer el conteo

#to , i, a, is, no son stop words, los tenemos q sacar

to           2234
i            2217
you          1921
a            1433
the          1326
u             996
and           968
is            868
in            857
my            755
for           704
your          677
of            614
me            611
have          568
call          556
on            521
are           487
that          470
it            466
2             457
so            423
but           422
or            415
not           411
at            400
can           386
ur            385
if            382
with          379
will          379
i'm           377
be            376
get           375
do            364
just          363
we            346
this          309
when          283
from          277
&lt;#&gt;     276
go            265
up            264
all           261
no            258
4             255
how           254
what          252
now           247
.             241
like          236
got           235
know          230
was           230
free          228
out       

In [15]:
stop=stopwords.words('english')

def remove_stopwords(message):
    if message is not None:
        words = message.strip().split()
        words_filtered = []
        for word in words:
            if word not in stop:
                words_filtered.append(word)
        result = " ".join(words_filtered)  
    else:
        result=None
    return result

In [16]:
df_interin = df_raw.copy()

In [17]:
df_interin['Message']=df_interin['Message'].apply(remove_stopwords)

In [18]:
df_interin['Message'].str.split(expand=True).stack().value_counts()[:60]

u            996
call         556
2            457
ur           385
i'm          377
get          375
&lt;#&gt;    276
go           265
4            255
.            241
like         236
got          235
know         230
free         228
come         220
good         201
?            187
send         187
want         183
text         175
time         169
i'll         168
...          163
love         163
going        161
ok           160
ü            157
need         157
r            153
still        151
one          150
txt          149
n            146
see          145
dont         140
new          136
tell         135
think        127
reply        126
mobile       124
take         124
back         121
stop         119
please       118
home         112
day          111
&            111
hi           108
claim        104
hope         103
make         101
give         100
pls           99
me.           98
phone         97
now.          94
later         94
happy         93
much          

In [23]:
df_interin['Message'] = df_interin['Message'].str.replace('.','',regex=False)
#df_interin['Message'] = df_interin['Message'].str.replace("""[\?\&\#\,\]""")
df_interin['Message']=df_interin['Message'].str.replace('''[\?\&\#,;ü']''','',regex=True)

In [24]:
df_interin['Message'].str.split(expand=True).stack().value_counts()[:60]

u         1105
call       573
2          472
im         461
ur         386
get        382
go         277
ltgt       276
ok         275
4          269
free       253
know       249
like       243
ill        238
got        237
good       232
come       228
time       205
want       193
send       190
love       189
text       186
day        183
going      170
one        170
me         167
need       166
lor        160
home       160
you        156
see        155
still      154
now        154
sorry      153
r          153
stop       150
back       150
txt        149
dont       147
n          146
reply      143
tell       137
new        136
later      134
think      132
mobile     128
today      128
hi         128
well       127
it         127
take       126
please     125
da         125
cant       124
phone      118
claim      111
night      111
much       109
dear       109
hey        107
dtype: int64

In [25]:
def normalize_string(text_string):
    if text_string is not None:
        result = unicodedata.normalize('NFD',text_string).encode('ascii','ignore').decode()
    else:
        result = None
    return result

df_interin['Message'] = df_interin['Message'].apply(normalize_string)

In [26]:
df_interin['Message'].str.split(expand=True).stack().value_counts()[:10]

u       1105
call     573
2        472
im       470
ur       386
get      382
go       277
ltgt     276
ok       275
4        269
dtype: int64

In [27]:
#saca repeticion de letras tipo looooveee a love
def replace_multiple_letters(message):
    if message is not None:
        result = re.sub(r'([a-zA-Z])\1{2,}',r'\1',message)
    else:
        result=None
    return result

In [28]:
df_interin['Message'] = df_interin['Message'].apply(replace_multiple_letters)
df_interin['Message'].str.split(expand=True).stack().value_counts()[:10]

u       1105
call     573
2        472
im       470
ur       386
get      382
go       277
ltgt     276
ok       275
4        269
dtype: int64

In [30]:
df = df_interin.copy()

X = df['Message']
y = df['Category']

#train and test
X_train, X_test, y_train, y_test = train_test_split(X,y, stratify=y, test_size=0.25, random_state=42)


In [31]:
# Vectorizo el texto de reviews a numeros 

vector = CountVectorizer(stop_words='english') #le saco las palabras de ingles
X_train = vector.fit_transform(X_train).toarray() #lo convierto en matriz
X_test = vector.transform(X_test).toarray() #

In [32]:
X_train

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [34]:
X_train.shape

(4179, 7553)

In [36]:
print(vector.get_feature_names_out())

['008704050406' '0089' '0121' ... 'zoe' 'zogtorius' 'zyada']


In [37]:
model = MultinomialNB()
model.fit(X_train, y_train)

MultinomialNB()

In [38]:
model.score(X_train, y_train) #hizo overfiting

0.9930605407992342

In [39]:
model.score(X_test, y_test)

0.9834888729361091

In [40]:
model.predict(vector.transform(['Hello my friend'])) # a ver q me dice si es spam o que, 

array(['ham'], dtype='<U4')

In [41]:
model.predict(vector.transform(['free for season'])) #largo spam!

array(['spam'], dtype='<U4')

In [43]:
def compress_date_street_name(message):
    if message is not None:
        expr = '(\d{1,2})\sde\s([ene|feb|mar|abr|may|jun|jul|ago|set|sep|oct|nov|dic].*)'
        result = re.sub(expr,r'\1de\2',message)
    else:
        result = None
    return result

compress_date_street_name('calle 18 de julio nro 3254 esq pepe carloncho')

'calle 18dejulio nro 3254 esq pepe carloncho'

In [42]:
#Grabo el modelo
import pickle

filename = '../models/nb_model.sav'
pickle.dump(model, open(filename,'wb'))